In [1]:
%%capture
!pip install torch-geometric

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
node_path = '/content/drive/MyDrive/node_classification/text_classification'

In [5]:
from torch_geometric.data import Data
import numpy as np
import torch

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
dataset = torch.load(f"{node_path}/data/ptg_complete_data.pth")
dataset.num_classes = len(torch.unique(dataset.y))

In [8]:
dataset = dataset.to(device)

In [9]:
dataset

Data(x=[50000, 384], edge_index=[2, 613827], y=[50000], num_classes=2, train_mask=[50000], val_mask=[50000], test_mask=[50000])

In [ ]:
dataset

In [10]:
import torch
from torch.nn import Linear
import torch.nn.functional as F


class MLP(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(12345)
        self.lin1 = Linear(dataset.num_features, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x):
        x = self.lin1(x)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

model = MLP(hidden_channels=16)
model = model.to(device)
print(model)

MLP(
  (lin1): Linear(in_features=384, out_features=16, bias=True)
  (lin2): Linear(in_features=16, out_features=2, bias=True)
)


In [11]:
data = dataset

In [16]:
from IPython.display import Javascript  # Restrict height of output cell.

model = MLP(hidden_channels=16)
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)  # Define optimizer.

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test():
      model.eval()
      out = model(data.x)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc

for epoch in range(1, 201):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 0.6982
Epoch: 002, Loss: 0.6897
Epoch: 003, Loss: 0.6817
Epoch: 004, Loss: 0.6729
Epoch: 005, Loss: 0.6622
Epoch: 006, Loss: 0.6482
Epoch: 007, Loss: 0.6351
Epoch: 008, Loss: 0.6219
Epoch: 009, Loss: 0.6095
Epoch: 010, Loss: 0.5944
Epoch: 011, Loss: 0.5787
Epoch: 012, Loss: 0.5684
Epoch: 013, Loss: 0.5565
Epoch: 014, Loss: 0.5476
Epoch: 015, Loss: 0.5364
Epoch: 016, Loss: 0.5284
Epoch: 017, Loss: 0.5235
Epoch: 018, Loss: 0.5176
Epoch: 019, Loss: 0.5125
Epoch: 020, Loss: 0.5043
Epoch: 021, Loss: 0.5041
Epoch: 022, Loss: 0.5012
Epoch: 023, Loss: 0.4972
Epoch: 024, Loss: 0.4940
Epoch: 025, Loss: 0.4896
Epoch: 026, Loss: 0.4883
Epoch: 027, Loss: 0.4856
Epoch: 028, Loss: 0.4840
Epoch: 029, Loss: 0.4826
Epoch: 030, Loss: 0.4800
Epoch: 031, Loss: 0.4775
Epoch: 032, Loss: 0.4774
Epoch: 033, Loss: 0.4770
Epoch: 034, Loss: 0.4745
Epoch: 035, Loss: 0.4743
Epoch: 036, Loss: 0.4720
Epoch: 037, Loss: 0.4699
Epoch: 038, Loss: 0.4696
Epoch: 039, Loss: 0.4683
Epoch: 040, Loss: 0.4705


In [17]:
test_acc = test()
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.7924


## GCN finetuning

---

In [18]:
from torch_geometric.nn import GCNConv


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x


In [19]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=16)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x, data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test():
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc


for epoch in range(1, 101):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Loss: 0.6938
Epoch: 002, Loss: 0.6827
Epoch: 003, Loss: 0.6714
Epoch: 004, Loss: 0.6580
Epoch: 005, Loss: 0.6450
Epoch: 006, Loss: 0.6312
Epoch: 007, Loss: 0.6164
Epoch: 008, Loss: 0.6007
Epoch: 009, Loss: 0.5864
Epoch: 010, Loss: 0.5746
Epoch: 011, Loss: 0.5636
Epoch: 012, Loss: 0.5551
Epoch: 013, Loss: 0.5458
Epoch: 014, Loss: 0.5384
Epoch: 015, Loss: 0.5320
Epoch: 016, Loss: 0.5259
Epoch: 017, Loss: 0.5213
Epoch: 018, Loss: 0.5155
Epoch: 019, Loss: 0.5121
Epoch: 020, Loss: 0.5102
Epoch: 021, Loss: 0.5062
Epoch: 022, Loss: 0.5053
Epoch: 023, Loss: 0.5025
Epoch: 024, Loss: 0.5010
Epoch: 025, Loss: 0.4968
Epoch: 026, Loss: 0.4982
Epoch: 027, Loss: 0.4927
Epoch: 028, Loss: 0.4924
Epoch: 029, Loss: 0.4917
Epoch: 030, Loss: 0.4903
Epoch: 031, Loss: 0.4911
Epoch: 032, Loss: 0.4905
Epoch: 033, Loss: 0.4864
Epoch: 034, Loss: 0.4851
Epoch: 035, Loss: 0.4842
Epoch: 036, Loss: 0.4846
Epoch: 037, Loss: 0.4848
Epoch: 038, Loss: 0.4836
Epoch: 039, Loss: 0.4845
Epoch: 040, Loss: 0.4815


In [20]:
test_acc = test()
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.7855


## GAT fineutning

---

In [21]:
from torch_geometric.nn import GATConv


class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GATConv(dataset.num_features, hidden_channels, heads=heads)  # TODO
        self.conv2 = GATConv(hidden_channels * heads, dataset.num_classes, heads=1)  # TODO

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [22]:
model = GAT(hidden_channels=16, heads=16)
model = model.to(device)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

GAT(
  (conv1): GATConv(384, 16, heads=16)
  (conv2): GATConv(256, 2, heads=1)
)


In [23]:
def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x, data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test(mask):
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      correct = pred[mask] == data.y[mask]  # Check against ground-truth labels.
      acc = int(correct.sum()) / int(mask.sum())  # Derive ratio of correct predictions.
      return acc

pacience = 0
las_val_acc = -1
total_pacience = 15
best_acc_test = -1
best_acc_val = -1
for epoch in range(1, 201):
    loss = train()
    val_acc = test(data.val_mask)
    test_acc = test(data.test_mask)

    if val_acc < las_val_acc:
      pacience += 1
    if val_acc > best_acc_val:
      best_acc_val =val_acc
    if test_acc > best_acc_test:
      best_acc_test = test_acc
    if pacience == total_pacience:
      break
    las_val_acc = val_acc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Loss: 0.6955, Val: 0.5008, Test: 0.5027
Epoch: 002, Loss: 0.7046, Val: 0.7024, Test: 0.7028
Epoch: 003, Loss: 0.6248, Val: 0.6540, Test: 0.6612
Epoch: 004, Loss: 0.6251, Val: 0.6776, Test: 0.6867
Epoch: 005, Loss: 0.6099, Val: 0.7208, Test: 0.7282
Epoch: 006, Loss: 0.5763, Val: 0.7276, Test: 0.7277
Epoch: 007, Loss: 0.5622, Val: 0.7168, Test: 0.7188
Epoch: 008, Loss: 0.5662, Val: 0.7236, Test: 0.7254
Epoch: 009, Loss: 0.5559, Val: 0.7360, Test: 0.7374
Epoch: 010, Loss: 0.5413, Val: 0.7412, Test: 0.7460
Epoch: 011, Loss: 0.5304, Val: 0.7428, Test: 0.7452
Epoch: 012, Loss: 0.5432, Val: 0.7448, Test: 0.7467
Epoch: 013, Loss: 0.5397, Val: 0.7492, Test: 0.7523
Epoch: 014, Loss: 0.5287, Val: 0.7488, Test: 0.7535
Epoch: 015, Loss: 0.5293, Val: 0.7444, Test: 0.7504
Epoch: 016, Loss: 0.5288, Val: 0.7456, Test: 0.7507
Epoch: 017, Loss: 0.5278, Val: 0.7504, Test: 0.7549
Epoch: 018, Loss: 0.5260, Val: 0.7516, Test: 0.7607
Epoch: 019, Loss: 0.5266, Val: 0.7512, Test: 0.7634
Epoch: 020, 

In [24]:
print(f'BEST Val: {best_acc_val:.4f}, BEST Test: {best_acc_test:.4f}')

BEST Val: 0.7620, BEST Test: 0.7684
